# Noise-Aware Classical Shadows with MEM

This notebook demonstrates QuartumSE's **noise-aware classical shadows (v1)** with **Measurement Error Mitigation (MEM)**.

## What You'll Learn
- How baseline shadows (v0) perform on ideal simulators
- How noise-aware shadows (v1) + MEM improve accuracy
- The automatic calibration workflow
- How to interpret diagnostics and provenance data

## What We've Built
✅ **Classical Shadows v0** - Baseline random Clifford measurements  
✅ **Classical Shadows v1** - Noise-aware with inverse channel correction  
✅ **MEM Integration** - Automatic confusion matrix calibration  
✅ **IBM Connector** - Vendor-neutral backend abstraction  
✅ **Shot Persistence** - Parquet storage with replay capability  
✅ **Full Provenance** - Reproducible experiments with manifests

## Setup

In [ ]:
import sys
from pathlib import Path

# Add QuartumSE to path if running from notebooks directory
sys.path.insert(0, str(Path.cwd().parent / "src"))

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import numpy as np

from quartumse import ShadowEstimator
from quartumse.shadows import ShadowConfig
from quartumse.shadows.config import ShadowVersion
from quartumse.shadows.core import Observable
from quartumse.reporting.manifest import MitigationConfig

print("✓ QuartumSE imported successfully")

## Create a GHZ State

GHZ states are maximally entangled: `|GHZ⟩ = (|000⟩ + |111⟩) / √2`

**Analytical expectations:**
- `⟨Z₀⟩ = 0` (qubit in superposition)
- `⟨Z₀Z₁⟩ = +1` (perfect correlation)
- `⟨Z₀Z₁Z₂⟩ = +1` (all qubits correlated)

In [ ]:
# Create 3-qubit GHZ state
circuit = QuantumCircuit(3)
circuit.h(0)  # Create superposition
circuit.cx(0, 1)  # Entangle qubit 0 and 1
circuit.cx(0, 2)  # Entangle qubit 0 and 2

print("GHZ Circuit:")
print(circuit)
print(f"\nDepth: {circuit.depth()}, Gates: {circuit.size()}")

## Define Observables

We'll measure:
- Single-qubit observables: `Z₀, Z₁, Z₂`
- Two-qubit correlations: `Z₀Z₁, Z₀Z₂`
- Three-qubit correlation: `Z₀Z₁Z₂`

In [ ]:
observables = [
    Observable("ZII", coefficient=1.0),  # ⟨Z₀⟩
    Observable("IZI", coefficient=1.0),  # ⟨Z₁⟩
    Observable("IIZ", coefficient=1.0),  # ⟨Z₂⟩
    Observable("ZZI", coefficient=1.0),  # ⟨Z₀Z₁⟩
    Observable("ZIZ", coefficient=1.0),  # ⟨Z₀Z₂⟩
    Observable("ZZZ", coefficient=1.0),  # ⟨Z₀Z₁Z₂⟩
]

# Analytical ground truth for GHZ
analytical = {
    "1.0*ZII": 0.0,
    "1.0*IZI": 0.0,
    "1.0*IIZ": 0.0,
    "1.0*ZZI": 1.0,
    "1.0*ZIZ": 1.0,
    "1.0*ZZZ": 1.0,
}

print("Observables to estimate:")
for obs in observables:
    print(f"  {obs} → Expected: {analytical[str(obs)]:+.1f}")

## Experiment 1: Baseline Classical Shadows (v0)

Standard classical shadows with **no noise mitigation**.

In [ ]:
print("="*70)
print("BASELINE CLASSICAL SHADOWS (v0)")
print("="*70)

# Configure v0 shadows
shadow_config_v0 = ShadowConfig(
    version=ShadowVersion.V0_BASELINE,
    shadow_size=200,
    random_seed=42,
    confidence_level=0.95,
)

# Create estimator with AerSimulator
estimator_v0 = ShadowEstimator(
    backend=AerSimulator(seed_simulator=123),
    shadow_config=shadow_config_v0,
    data_dir="./notebook_data"
)

print(f"Shadow implementation: {type(estimator_v0.shadow_impl).__name__}")
print(f"Shadow size: {shadow_config_v0.shadow_size}")
print(f"Backend: {estimator_v0.backend.name}")
print()

In [ ]:
# Run estimation
result_v0 = estimator_v0.estimate(
    circuit=circuit,
    observables=observables,
    save_manifest=True
)

print("\nResults (v0):")
print(f"{'Observable':<15} {'Estimated':<12} {'Expected':<12} {'Error':<12} {'CI Width'}")
print("-"*65)

errors_v0 = []
for obs in observables:
    obs_str = str(obs)
    estimated = result_v0.observables[obs_str]["expectation_value"]
    expected = analytical[obs_str]
    error = abs(estimated - expected)
    ci_width = result_v0.observables[obs_str]["ci_width"]
    errors_v0.append(error)
    
    print(f"{obs_str:<15} {estimated:>10.4f}  {expected:>10.1f}  {error:>10.4f}  {ci_width:>10.4f}")

print(f"\nMean Absolute Error (v0): {np.mean(errors_v0):.4f}")
print(f"Manifest saved: {result_v0.manifest_path}")
print(f"Shot data saved: {result_v0.shot_data_path}")

## Experiment 2: Noise-Aware Shadows with MEM (v1)

Now with **automatic MEM calibration** and **noise-aware corrections**.

### What Happens Automatically:
1. **MEM calibrates** a confusion matrix by preparing all 2³=8 basis states
2. **Confusion matrix is inverted** to correct measurement errors
3. **Each shadow measurement** is corrected before reconstruction
4. **Provenance tracks** the MEM technique in the manifest

In [ ]:
print("="*70)
print("NOISE-AWARE CLASSICAL SHADOWS (v1) with MEM")
print("="*70)

# Configure v1 noise-aware shadows
shadow_config_v1 = ShadowConfig(
    version=ShadowVersion.V1_NOISE_AWARE,
    shadow_size=200,  # Same as v0 for fair comparison
    random_seed=42,
    confidence_level=0.95,
    apply_inverse_channel=True,
)

# Configure MEM with calibration budget
mitigation_config = MitigationConfig(
    techniques=[],  # Will be auto-populated
    parameters={"mem_shots": 1024}  # Shots for calibration
)

# Create estimator
estimator_v1 = ShadowEstimator(
    backend=AerSimulator(seed_simulator=123),
    shadow_config=shadow_config_v1,
    mitigation_config=mitigation_config,
    data_dir="./notebook_data"
)

print(f"Shadow implementation: {type(estimator_v1.shadow_impl).__name__}")
print(f"Shadow size: {shadow_config_v1.shadow_size}")
print(f"MEM calibration shots: {mitigation_config.parameters['mem_shots']}")
print(f"Has MEM: {estimator_v1.measurement_error_mitigation is not None}")
print()

In [ ]:
# Run estimation (MEM calibrates automatically)
print("Running estimation with automatic MEM calibration...\n")

result_v1 = estimator_v1.estimate(
    circuit=circuit,
    observables=observables,
    save_manifest=True
)

print("\nResults (v1 + MEM):")
print(f"{'Observable':<15} {'Estimated':<12} {'Expected':<12} {'Error':<12} {'CI Width'}")
print("-"*65)

errors_v1 = []
for obs in observables:
    obs_str = str(obs)
    estimated = result_v1.observables[obs_str]["expectation_value"]
    expected = analytical[obs_str]
    error = abs(estimated - expected)
    ci_width = result_v1.observables[obs_str]["ci_width"]
    errors_v1.append(error)
    
    print(f"{obs_str:<15} {estimated:>10.4f}  {expected:>10.1f}  {error:>10.4f}  {ci_width:>10.4f}")

print(f"\nMean Absolute Error (v1): {np.mean(errors_v1):.4f}")
print(f"Manifest saved: {result_v1.manifest_path}")

## Verification: MEM Pipeline

Let's verify that MEM was actually applied:

In [ ]:
print("MEM Pipeline Verification:")
print("-" * 50)

# Check confusion matrix
mem = estimator_v1.measurement_error_mitigation
print(f"✓ Confusion matrix shape: {mem.confusion_matrix.shape}")
print(f"✓ Confusion matrix (should be ≈ identity for ideal simulator):")
print(np.round(mem.confusion_matrix, 3))
print()

# Check noise-corrected distributions
distributions = estimator_v1.shadow_impl.noise_corrected_distributions
print(f"✓ Noise-corrected distributions computed: {distributions is not None}")
if distributions is not None:
    print(f"✓ Distributions shape: {distributions.shape}")
    print(f"  (200 shadows × 8 possible states for 3 qubits)")
print()

# Check mitigation config
print(f"✓ MEM in mitigation techniques: {'MEM' in estimator_v1.mitigation_config.techniques}")
print(f"✓ Calibrated qubits: {mem._calibrated_qubits}")

## Comparison: v0 vs v1

Even on an ideal simulator, noise-aware shadows demonstrate the correct workflow.

In [ ]:
print("="*70)
print("COMPARISON: v0 (Baseline) vs v1 (Noise-Aware + MEM)")
print("="*70)
print()

print(f"Mean Absolute Error:")
print(f"  v0 (Baseline):        {np.mean(errors_v0):.4f}")
print(f"  v1 (Noise-Aware):     {np.mean(errors_v1):.4f}")
print()

improvement = (np.mean(errors_v0) - np.mean(errors_v1)) / np.mean(errors_v0) * 100
if improvement > 0:
    print(f"✓ Improvement: {improvement:.1f}% reduction in error")
else:
    print(f"  Similar performance (expected on ideal simulator)")
print()

print("Key Differences:")
print("  v0: Direct shadow reconstruction from raw measurements")
print("  v1: MEM-corrected probability distributions + marginalization")
print()
print("On real hardware with noise, v1 would show significant improvement!")

## Provenance & Reproducibility

All experiments are fully reproducible via provenance manifests:

In [ ]:
from quartumse.reporting.manifest import ProvenanceManifest

# Load v1 manifest
manifest = ProvenanceManifest.from_json(result_v1.manifest_path)

print("Provenance Manifest (v1):")
print("-" * 50)
print(f"Experiment ID: {manifest.schema.experiment_id[:16]}...")
print(f"Backend: {manifest.schema.backend.backend_name}")
print(f"Circuit hash: {manifest.schema.circuit.circuit_hash[:16]}...")
print(f"Shadow version: {manifest.schema.shadows.version}")
print(f"Mitigation techniques: {manifest.schema.mitigation.techniques}")
print(f"MEM shots: {manifest.schema.mitigation.parameters.get('mem_shots', 'N/A')}")
print(f"Total shots: {manifest.schema.resource_usage.total_shots:,}")
print(f"Execution time: {manifest.schema.resource_usage.execution_time_seconds:.2f}s")
print(f"QuartumSE version: {manifest.schema.quartumse_version}")
print()
print(f"✓ Full reproducibility guaranteed via manifest + shot data")

## Summary: What We've Achieved

### ✅ Technical Accomplishments

1. **Classical Shadows v0 (Baseline)**
   - Random local Clifford measurements
   - Confidence intervals via bootstrapping
   - Variance bounds

2. **Classical Shadows v1 (Noise-Aware)**
   - Automatic MEM calibration
   - Confusion matrix inversion
   - Probability distribution corrections
   - Marginalized expectation values

3. **Measurement Error Mitigation (MEM)**
   - Computational basis calibration
   - Matrix inversion with pseudo-inverse fallback
   - Physical constraint enforcement

4. **IBM Quantum Connector**
   - Vendor-neutral backend abstraction
   - Calibration snapshot extraction
   - Graceful fallback to local simulators

5. **Shot Data Persistence**
   - Parquet storage format
   - Replay capability for new observables
   - Automatic diagnostics computation

6. **Full Provenance Tracking**
   - JSON manifests with complete experiment context
   - Circuit fingerprints
   - Backend calibration snapshots
   - Version tracking

### 📊 Value Proposition

**For Researchers:**
- Reproducible quantum experiments with full provenance
- "Measure once, ask later" workflow saves hardware costs
- Automatic noise mitigation without manual tuning

**For Industry:**
- Vendor-neutral platform (IBM, AWS, etc.)
- Cost optimization via shot-efficient estimation
- Auditable results for compliance

**Scientific Impact:**
- Implements cutting-edge classical shadows theory (Huang et al. 2020)
- Bridges gap between theory and practical implementation
- Enables hardware validation of shadow methods

### 🎯 Phase 1 Status

**Completed:**
- ✅ Classical Shadows v0 + v1
- ✅ MEM integration
- ✅ IBM connector
- ✅ Shot persistence + replay
- ✅ Provenance infrastructure
- ✅ S-T01 + S-T02 experiments

**Next:**
- Run S-T01/S-T02 to validate SSR ≥ 1.2× target
- Complete C/O/B/M starter experiments
- Run on real IBM hardware for noise validation